This notebook looks at the output from the snippet repository
and how to use it to train NER, classification, and mlm models.

In [1]:

from typing import List
from spacy import displacy
import src.data.snippet_repository as sr

### Named Enitiy Recognition Models (NER)

In [2]:
ner_repo = sr.SnippetRepository(sr.SnippetRepositoryMode.NER)

In [3]:
ner_data = ner_repo.get_training_data(batch_size=10)
detected = False
while not detected:
    ner_df = next(ner_data)
    detected = any(ner_df['ner_tags'].apply(lambda ner_tags: any(map(lambda t: t!="O", ner_tags))))

In [4]:
text = ner_df.iloc[9].text
ner_tags = ner_df.iloc[9].ner_tags
sr.visualize_ner_tags(text, ner_tags)

In [6]:
# load the model here

from transformers import AutoConfig, AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline



In [8]:
AutoConfig.from_pretrained("dslim/bert-base-NER")

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

BertConfig {
  "_name_or_path": "dslim/bert-base-NER",
  "_num_labels": 9,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-MISC",
    "2": "I-MISC",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 7,
    "B-MISC": 1,
    "B-ORG": 5,
    "B-PER": 3,
    "I-LOC": 8,
    "I-MISC": 2,
    "I-ORG": 6,
    "I-PER": 4,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size"

In [11]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
config = AutoConfig.from_pretrained(
    "dslim/bert-base-NER", 
    output_attentions=True,
    output_hidden_states=True,
)
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER", config=config)

In [22]:
features = tokenizer(
    "This is sample text and it is grand".split(), 
    is_split_into_words=True,
    return_tensors="pt",
)
features

{'input_ids': tensor([[ 101, 1188, 1110, 6876, 3087, 1105, 1122, 1110, 5372,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [23]:
outs = model(**features)

In [24]:
outs.hidden_states[0].shape, outs.attentions[0].shape

(torch.Size([1, 10, 768]), torch.Size([1, 12, 10, 10]))

In [25]:
outs.attentions[0][0, 0,...]

tensor([[5.0984e-01, 1.7902e-02, 4.4900e-03, 7.2620e-03, 8.8475e-03, 5.9365e-03,
         2.4017e-02, 5.7133e-03, 1.8693e-02, 3.9729e-01],
        [1.5214e-01, 8.2471e-02, 1.0092e-01, 2.0781e-01, 1.0100e-01, 6.3148e-03,
         8.7182e-02, 9.8551e-02, 7.8215e-02, 8.5383e-02],
        [5.3224e-02, 5.3401e-02, 2.6654e-01, 2.0507e-01, 6.4948e-02, 2.5070e-02,
         6.6980e-02, 2.0637e-01, 2.1779e-02, 3.6618e-02],
        [2.1338e-01, 1.1654e-01, 1.0632e-01, 1.4516e-01, 1.0935e-01, 1.4510e-02,
         8.8916e-02, 1.2407e-01, 6.2667e-02, 1.9075e-02],
        [4.0572e-02, 9.8657e-02, 6.6175e-02, 2.5272e-01, 1.2026e-01, 5.1738e-02,
         1.1349e-01, 6.9439e-02, 1.6019e-01, 2.6759e-02],
        [1.1624e-01, 1.3273e-01, 6.0380e-02, 1.8915e-01, 8.1651e-02, 6.3257e-02,
         1.1841e-01, 5.4545e-02, 9.2658e-02, 9.0974e-02],
        [8.4839e-02, 1.2499e-01, 9.5169e-02, 2.3637e-01, 7.8517e-02, 3.1685e-02,
         1.2090e-01, 7.2664e-02, 5.8738e-02, 9.6132e-02],
        [4.1770e-02, 5.5790